In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [76]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [23]:
def obtain_links_periodos(driver):
    links_asistencias = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('asistencias', link)):
            links_asistencias.append(link)
    return(links_asistencias)

def obtain_links_days(driver):
    links_asistencias_sesion = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('asisteinifinsesion', link)):
            links_asistencias_sesion.append(link)
    return(links_asistencias_sesion)

def obtain_partyAttendance(driver):
    links_asistencias_sesion = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('listados_asistencias', link)):
            links_asistencias_sesion.append(link)
    return(links_asistencias_sesion)

In [47]:
def month_to_num(x):
    if x == 'Enero':
        return(1)
    elif x == 'Febrero':
        return(2)
    elif x == 'Marzo':
        return(3)
    elif x == 'Abril':
        return(4)
    elif x == 'Mayo':
        return(5)
    elif x == 'Junio':
        return(6)
    elif x == 'Julio':
        return(7)
    elif x == 'Agosto':
        return(8)
    elif x == 'Septiembre':
        return(9)
    elif x == 'Octubre':
        return(10)
    elif x == 'Noviembre':
        return(11)
    elif x == 'Diciembre':
        return(12)

In [63]:
import unidecode

In [62]:
def obtain_attendance(driver, legislatura):
    temp = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(1) > td > b > u').text
    date_session = datetime.date(int(temp.split('-')[2]), month_to_num(temp.split('-')[1]), int(temp.split('-')[0]))
    party_session = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > span > b').text
    
    
    year_legis = []
    month_legis = []
    day_legis = []
    members = []
    party = []
    type_at = []
    legis = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('tr')[9:]:
        try:
            if any(re.findall('[a-z]+', i.find_elements_by_css_selector('td')[0].text)):
                continue
            else:
                members.append(unidecode.unidecode(i.find_elements_by_css_selector('td')[1].text))
                type_at.append(i.find_elements_by_css_selector('td')[2].text)
                
                party.append(party_session)
                year_legis.append(date_session.year)
                month_legis.append(date_session.month)
                day_legis.append(date_session.day)
                legis.append(legislatura)
        except:
            continue
            
    return(pd.DataFrame({'legislatura':legis, 
                 'year_session':year_legis, 
                 'month_session':month_legis, 
                 'day_session':day_legis, 
                 'member':members, 
                 'party':party, 
                 'type_attendance':type_at
                 }))

    

In [67]:
link_legislaturas =[
    'http://sitllx.diputados.gob.mx/asistencias_diputados_todosnp.php', 
    'http://sitllxi.diputados.gob.mx/asistencias_diputados_todosnplxi.php', 
    'http://sitllxiii.diputados.gob.mx/asistencias_diputados_todosnplxiii.ph', 
    'http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_todosnplxiv.php'
]

In [ ]:
for link, leg in tqdm(zip(link_legislaturas, [60, 61, 63, 64]), total = 4):
    data_legislatura = []
    driver.get(link)
    links_periodos = obtain_links_periodos(driver)
    for link2 in tqdm(links_periodos):
        driver.get(link2)
        links_sesiones = obtain_links_days(driver)
        for link3 in tqdm(links_sesiones):
            driver.get(link3)
            links_sesiones_partido = obtain_partyAttendance(driver)
            for link4 in tqdm(links_sesiones_partido):
                driver.get(link4)
                data_legislatura.append(obtain_attendance(driver, 63))
    df_attendance_leg = pd.concat(data_legislatura)
    name_file = 'floor_attendance' + str(leg) + '.csv'
    df_attendance_leg.to_csv('../../data/00-id_data/' + name_file, encoding = 'utf-8')

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:34<04:32, 34.00s/it]


 22%|██▏       | 2/9 [00:57<03:35, 30.84s/it]


 33%|███▎      | 3/9 [01:19<02:48, 28.11s/it]


 44%|████▍     | 4/9 [01:24<01:46, 21.37s/it]


 56%|█████▌    | 5/9 [01:29<01:05, 16.25s/it]


 67%|██████▋   | 6/9 [01:31<00:36, 12.17s/it]


 78%|███████▊  | 7/9 [01:37<00:20, 10.13s/it]


 89%|████████▉ | 8/9 [01:40<00:07,  8.00s/it]


100%|██████████| 9/9 [01:42<00:00, 11.36s/it]


  3%|▎         | 1/35 [01:42<58:10, 102.66s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:51<06:50, 51.31s/it]


 22%|██▏       | 2/9 [01:16<05:03, 43.33s/it]


 33%|███▎      | 3/9 [01:33<03:33, 35.64s/it]


 44%|████▍     | 4/9 [01:37<02:09, 25.94s/it]


 56%|█████▌    | 5/9 [01:39<01:15, 18.98s/it]


 67%|██████▋   | 6/9 [01:41<00:41, 13.91s/it]


 78%|███████▊  | 7/9 [01:43<00:20, 10.32s/it]


 89%|████████▉ | 8/9 [

  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:34<04:32, 34.03s/it]


 22%|██▏       | 2/9 [01:05<03:53, 33.35s/it]


 33%|███▎      | 3/9 [01:39<03:20, 33.40s/it]


 44%|████▍     | 4/9 [01:43<02:02, 24.56s/it]


 56%|█████▌    | 5/9 [01:46<01:12, 18.15s/it]


 67%|██████▋   | 6/9 [01:48<00:40, 13.34s/it]


 78%|███████▊  | 7/9 [01:50<00:19,  9.85s/it]


 89%|████████▉ | 8/9 [01:51<00:07,  7.28s/it]


100%|██████████| 9/9 [01:52<00:00, 12.49s/it]


 40%|████      | 14/35 [23:24<36:50, 105.27s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:51<06:48, 51.09s/it]


 22%|██▏       | 2/9 [01:13<04:57, 42.53s/it]


 33%|███▎      | 3/9 [01:36<03:39, 36.61s/it]


 44%|████▍     | 4/9 [01:42<02:16, 27.34s/it]


 56%|█████▌    | 5/9 [01:46<01:22, 20.53s/it]


 67%|██████▋   | 6/9 [01:50<00:46, 15.37s/it]


 78%|███████▊  | 7/9 [01:52<00:22, 11.37s/it]


 89%|████████▉ | 8/9 [01:53<00:08,  8.41s/it]


100%|██████████| 9/9 [01:54<00:00, 12.74s/it]


 43%|

 22%|██▏       | 2/9 [00:55<03:25, 29.42s/it]


 33%|███▎      | 3/9 [01:11<02:31, 25.30s/it]


 44%|████▍     | 4/9 [01:14<01:33, 18.66s/it]


 56%|█████▌    | 5/9 [01:17<00:56, 14.05s/it]


 67%|██████▋   | 6/9 [01:19<00:31, 10.45s/it]


 78%|███████▊  | 7/9 [01:21<00:15,  7.76s/it]


 89%|████████▉ | 8/9 [01:22<00:05,  5.83s/it]


100%|██████████| 9/9 [01:23<00:00,  9.29s/it]


 77%|███████▋  | 27/35 [43:23<11:22, 85.35s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:34<04:39, 35.00s/it]


 22%|██▏       | 2/9 [00:56<03:37, 31.08s/it]


 33%|███▎      | 3/9 [01:16<02:44, 27.50s/it]


 44%|████▍     | 4/9 [01:20<01:42, 20.51s/it]


 56%|█████▌    | 5/9 [01:24<01:03, 15.76s/it]


 67%|██████▋   | 6/9 [01:28<00:36, 12.14s/it]


 78%|███████▊  | 7/9 [01:31<00:18,  9.40s/it]


 89%|████████▉ | 8/9 [01:33<00:07,  7.02s/it]


100%|██████████| 9/9 [01:34<00:00, 10.50s/it]


 80%|████████  | 28/35 [44:58<10:17, 88.22s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%

 33%|███▎      | 3/9 [01:14<02:44, 27.34s/it]


 44%|████▍     | 4/9 [01:19<01:43, 20.64s/it]


 56%|█████▌    | 5/9 [01:23<01:02, 15.74s/it]


 67%|██████▋   | 6/9 [01:26<00:35, 11.81s/it]


 78%|███████▊  | 7/9 [01:28<00:17,  8.88s/it]


 89%|████████▉ | 8/9 [01:29<00:06,  6.62s/it]


100%|██████████| 9/9 [01:30<00:00, 10.09s/it]


 19%|█▉        | 5/26 [07:31<31:21, 89.59s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:40<05:20, 40.04s/it]


 22%|██▏       | 2/9 [01:08<04:16, 36.60s/it]


 33%|███▎      | 3/9 [01:26<03:05, 30.90s/it]


 44%|████▍     | 4/9 [01:29<01:53, 22.73s/it]


 56%|█████▌    | 5/9 [01:33<01:07, 16.97s/it]


 67%|██████▋   | 6/9 [01:35<00:37, 12.53s/it]


 78%|███████▊  | 7/9 [01:37<00:18,  9.43s/it]


 89%|████████▉ | 8/9 [01:38<00:06,  6.93s/it]


100%|██████████| 9/9 [01:39<00:00, 11.10s/it]


 23%|██▎       | 6/26 [09:11<30:56, 92.81s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:37<05:00, 37.54s/it]


 22%|█

 56%|█████▌    | 5/9 [04:18<02:59, 44.96s/it]


 67%|██████▋   | 6/9 [04:26<01:42, 34.09s/it]


 78%|███████▊  | 7/9 [04:32<00:51, 25.55s/it]


 89%|████████▉ | 8/9 [04:35<00:18, 18.83s/it]


100%|██████████| 9/9 [04:38<00:00, 30.92s/it]


 69%|██████▉   | 18/26 [47:03<34:25, 258.24s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [02:19<18:36, 139.51s/it]


 22%|██▏       | 2/9 [03:20<13:31, 115.94s/it]


 33%|███▎      | 3/9 [04:45<10:39, 106.60s/it]


 44%|████▍     | 4/9 [05:03<06:40, 80.15s/it] 


 56%|█████▌    | 5/9 [05:15<03:58, 59.56s/it]


 67%|██████▋   | 6/9 [05:21<02:11, 43.67s/it]


 78%|███████▊  | 7/9 [05:27<01:04, 32.25s/it]


 89%|████████▉ | 8/9 [05:31<00:23, 23.87s/it]


100%|██████████| 9/9 [05:35<00:00, 37.24s/it]


 73%|███████▎  | 19/26 [52:39<32:50, 281.53s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [01:30<12:03, 90.45s/it]

In [73]:
link4

'http://sitllx.diputados.gob.mx/listados_asistenciasnp.php?partidot=2&sesiont=18'

In [35]:
driver.find_element_by_css_selector('table').find_elements_by_css_selector('tr')[10].find_elements_by_css_selector('td')

[<selenium.webdriver.remote.webelement.WebElement (session="5a7983def97e5f5ee67722be4e0f3abc", element="398fb220-661d-4541-982e-0127b71cf24a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5a7983def97e5f5ee67722be4e0f3abc", element="7d2fb9bc-2f5f-4e21-b5c8-b7aac6802344")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5a7983def97e5f5ee67722be4e0f3abc", element="b4a992c1-f099-450a-b774-5afe41e78233")>]

In [28]:
for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
    link = i.get_attribute('href')
    if any(re.findall('asistencias_por_', link)):
        print(i.text)

Abdala Carmona Yahleel
Abdala Saad Antonio Tarek
Abramo Masso Yerico
Aguilar Robles David
Aguilar Yunes Marco Antonio
Aguirre Chávez Marco Polo
Aguirre Romero Andrés
Alcántara Rojas María Guadalupe
Almanza Monroy Fidel
Alpizar Carrillo Lucely Del Perpetuo Socorro
Alvarado Varela Edith Anabel
Álvarez Ortiz Héctor Javier
Amaro Cancino Antonio
Anaya Mota Claudia Edith
Anaya Orozco Alfredo
Antelo Esper Bernardino
Arcos Velázquez Montserrat Alicia
Arellano Núñez Efrain
Armenta Mier Alejandro
Arroyo Bello Erika Lorena
Barragán Amador Carlos
Barrientos Cano Laura Mitzi
Basáñez García Pablo
Bañales Arambula Ramón
Bedolla López Pablo
Bejos Nicolás Alfredo
Beltrones Sánchez Sylvana
Benítez Tiburcio Mariana
Bernal Casique Iveth
Boone Godoy Ana María
Botello Santibáñez María Bárbara
Bugarín Rodríguez Jasmine María
Cabrera Ruiz José Hugo
Camacho César
Camargo Félix María Esther Guadalupe
Canales Najjar Tristán Manuel
Carvallo Delfín Jorge Alejandro
Casillas Zanatta Gabriel
Castillo Martínez Edgar
C

In [12]:
driver.get(links[0])